# Creating a synthetic Q&A dataset

In [1]:
import os

proxy = 'http://dell-1.star:7890' # 3090 docker
# proxy = 'http://127.0.0.1:7890' # clash
# proxy = 'http://127.0.0.1:1080' # naiveproxy

os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

## Read in data, and create a context

In [2]:
import pandas as pd
df = pd.read_csv("docx/qa/total.csv")
df['context'] = "summary: " +  df.summary + "\n" +  "content: " + df.content
df.head()

,summary,content,summary_token_num,content_token_num,token_num,context
0,电信业务#电信业务分类目录中的基础电信业务有哪些,答案要点：(一)固定网络国内长途及本地电话业务；(二)移动网络电话和数据业务；(三)卫星通信...,13,104,117,summary: 电信业务#电信业务分类目录中的基础电信业务有哪些\ncontent: 答案...
1,电信业务#电信业务分类目录中的增值电信业务有哪些,答案要点：(一)电子邮件；(二)语音信箱；（三）在线信息库存储和检索；(四)电子数据交换；(...,13,66,79,summary: 电信业务#电信业务分类目录中的增值电信业务有哪些\ncontent: 答案...
2,电信业务#怎样连上Internet,答案要点：一般说来，连入Internet主要有异步拨号和专线接入这两种方式。异步方式是指通过...,8,118,126,summary: 电信业务#怎样连上Internet\ncontent: 答案要点：一般说来...
3,电信业务#何为ＥＣ（电子商务,答案要点：究竟什么是电子商务，不同的人往往有不同的理解。电子商务最通常的一个定义是：交易所涉...,8,74,82,summary: 电信业务#何为ＥＣ（电子商务\ncontent: 答案要点：究竟什么是电子...
4,电信业务#如何在网络中共享打印机,"答案要点：单击""开始""，指向""设置""，然后单击""打印机""。单击要共享的打印机。在""文件""菜单...",9,45,54,"summary: 电信业务#如何在网络中共享打印机\ncontent: 答案要点：单击""开始..."


## Create questions based on the context

In [3]:
import openai
openai.api_key_path = ".openai-key"

In [4]:
import requests

# requests.get("http://google.com")

context = """
summary: 网速慢的处理方法#检查是否硬盘空间不够，删除一些临时文件#MODE旁边有高功率的电器设备，很容易导致信号受干扰#建议用户在自己的计算机上安装杀毒软件，及时发现和查杀病毒•
content: 第八章建立连接正常但无法收发E-mail此情况一般是电脑设置问题。您可先了解一下，如果是其他ADSL用户可发，您不能发，请检查您的电脑如检查您是否设置了SmtP认证、发送邮件（SMTP服务器等如个别用户只可发小文件，不能发大文件，是因大多数E-mail服务器对附件的大小有限制。因此，在软件上应对较大的附件进行分拆设置，具体操作，可见E-mail软件的相关帮助
"""

In [27]:

response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=f"请根据以下文本生成问题，尽可能使用简体中文，问题表述需要清晰详细\n\n文本: {context}\n\n问题:\n1.",
            temperature=0,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
print(response.choices[0].text)

questions = "1." + response.choices[0].text
resp = openai.Completion.create(
            engine="text-davinci-003",
            prompt=f"根据以下文本生成问题的答案，尽可能使用简体中文\n\n文本: {context}\n\n问题:\n{questions}\n\n答案:\n1.",
            temperature=0,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
print(resp['choices'][0]['text'])

 如何检查硬盘空间是否不够？
2. 为什么MODE旁边有高功率的电器设备会导致信号受干扰？
3. 如何在自己的计算机上安装杀毒软件？
4. 如何检查电脑是否设置了SmtP认证、发送邮件（SMTP服务器）？
5. 为什么有些用户只可发小文件，不能发大文件？
6. 如何对较大的附件进行分拆设置？
 可以使用计算机的磁盘管理工具查看硬盘空间是否不够。
2. 高功率的电器设备会产生电磁干扰，影响MODE的信号传输。
3. 可以下载安装杀毒软件，然后根据提示进行安装。
4. 可以在计算机的网络设置中查看是否设置了SmtP认证、发送邮件（SMTP服务器）。
5. 大多数E-mail服务器对附件的大小有限制，因此有些用户只可发小文件，不能发大文件。
6. 可以在E-mail软件的设置中，对较大的附件进行分拆设置。


In [10]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": f"请根据以下文本生成问题，尽可能使用简体中文，问题表述需要清晰详细\n\n文本: {context}\n\n问题:\n1."
        }
    ],
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response.choices[0].message.content)

questions = "1." + response.choices[0].message.content
resp = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": f"根据以下文本生成问题的答案，尽可能使用简体中文\n\n文本: {context}\n\n问题:\n{questions}\n\n答案:\n1."
        }
    ],
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(resp.choices[0].message.content)

网速慢的处理方法有哪些？
2. 为什么MODE旁边的电器设备会影响信号？
3. 建议用户在自己的计算机上安装什么软件？
4. 电脑设置问题可能导致什么情况？
5. 如何解决不能发送大文件的问题？
检查是否硬盘空间不够，删除一些临时文件；建议用户在自己的计算机上安装杀毒软件，及时发现和查杀病毒。
2. 高功率的电器设备容易导致信号受干扰。
3. 建议用户在自己的计算机上安装杀毒软件，及时发现和查杀病毒。
4. 电脑设置问题可能导致无法收发E-mail。
5. 对较大的附件进行分拆设置，具体操作可见E-mail软件的相关帮助。


In [7]:
# use davinci 
def get_questions(context):
    try:
        response = openai.Completion.create(
            # engine="davinci-instruct-beta-v3",
            engine="text-davinci-003",
            prompt=f"请根据以下文本生成问题,尽可能使用简体中文\n\n文本: {context}\n\n问题:\n1.",
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
        return response['choices'][0]['text']
    except:
        return ""

df['questions']= df.context.apply(get_questions)
df['questions'] = "1." + df.questions

In [11]:
# use gpt-3.5-turbo
def get_questions_by_chat(context):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "user",
                    "content": f"请根据以下文本生成问题，尽可能使用简体中文，问题表述需要清晰详细\n\n文本: {context}\n\n问题:\n1."
                }
            ],
            temperature=0,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
        return response.choices[0].message.content
    except:
        return ""
df['questions']= df.context.apply(get_questions_by_chat)
df['questions'] = "1." + df.questions

In [8]:
print(df[['questions']].values)

[['1. 电信业务分类目录中的基础电信业务有哪些？\n2. 固定网络国内长途及本地电话业务属于哪类电信业务？\n3. 移动网络电话和数据业务包括哪些内容？\n4. 卫星通信及卫星移动通信业务有哪些特点？\n5. 互联网及其它公共数据传送业务有哪些优势？\n6. 带宽、波']
 ['1. 电信业务分类目录中的增值电信业务有哪些？\n2. 增值电信业务包括哪些内容？']
 ['1. 电信业务中连接Internet的方式有哪些？\n2. 异步拨号和专线接入有什么区别？\n3. 哪种方式更适合普通用户？\n4. 专线接入方式投资大，但是有什么优势？']
 ['1. 什么是电子商务（EC）？\n2. 电子商务的定义是什么？\n3. 电子商务的本质特征是什么？\n4. 电子商务活动是如何改变传统的商务服务模式的？']
 ['1. 如何在网络中共享打印机？\n2. 如何单击要共享的打印机？\n3. 如何在“文件”菜单上单击“属性”？\n4. 如何在“共享”选项卡上单击“共享为”？']
 ['1. 如何查看自己的IP地址？\n2. Win9X系统自带的WINIPCFG.EXE程序在哪里？']
 ['1. GSM是什么的缩写？\n2. GSM有哪些特点？\n3. GSM能够提供哪些增值业务？\n4. GSM为客户带来了什么方便？']
 ['1. GSM业务是什么？\n2. SIM卡是什么？\n3. SIM卡是由什么制成的？']
 ['1. GSM业务中SIM卡的主要作用是什么？\n2. SIM卡有哪些功能？\n3. SIM卡的身份识别功能有何作用？\n4. SIM卡的安全保密功能有何作用？\n5. SIM卡的基本信息存储功能有何作用？\n6. SIM卡的鉴权功能有何作用？']
 ['1. SIM卡ICCID有多少位？\n2. SIM卡ICCID的格式是什么？\n3. 中国国家代码是多少？\n4. 年份的代码是什么？\n5. 存储容量的单位是什么？']
 ['1. 什么是PIN1、PIN2和PUK码？\n2. 新购卡入网的用户PIN码是多少？\n3. 为什么中国联通不向客户公开PIN2码？\n4. 如果用户输入10次错误的PUK码，会发生什么？\n5. 如果SIM卡被锁住，应该如何解锁？\n6. 如果用户漫游在外地时SIM卡被锁，应该如何处理？']
 ['1. 什么是CDMA？\n2

## Create answers based on the context

In [9]:
def get_answers(row):
    try:
        response = openai.Completion.create(
            # engine="davinci-instruct-beta-v3",
            engine="text-davinci-003",
            prompt=f"根据以下文本生成问题的答案,尽可能使用简体中文\n\n文本: {row.context}\n\n问题:\n{row.questions}\n\n答案:\n1.",
            temperature=0,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response['choices'][0]['text']
    except Exception as e:
        print (e)
        return ""


df['answers']= df.apply(get_answers, axis=1)
df['answers'] = "1." + df.answers

This model's maximum context length is 4097 tokens, however you requested 4671 tokens (4171 in your prompt; 500 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 4097 tokens, however you requested 4773 tokens (4273 in your prompt; 500 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 4097 tokens, however you requested 4931 tokens (4431 in your prompt; 500 for the completion). Please reduce your prompt; or completion length.


In [ ]:
def get_answers_by_chat(row):
    try:
        response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages=[
                {
                    "role": "user",
                    "content": f"根据以下文本生成问题的答案，尽可能使用简体中文\n\n文本: {row.context}\n\n问题:\n{row.questions}\n\n答案:\n1."
                }
            ],
            temperature=0,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response.choices[0].message.content
    except Exception as e:
        print (e)
        return ""

df['answers']= df.apply(get_answers_by_chat, axis=1)
df['answers'] = "1." + df.answers

In [22]:
print(df[['answers']].values)

[['1. 中国联通手机营业厅分为网页版和客户端。\n2. 客户端提供7×24小时服务。\n3. 目前功能包括话费查询、交费充值、3G套餐变更、业务查询、积分查询、密码修改、积分兑换、网点查询、手机号码归属地、3G网络覆盖查询、PUK码查询、3G专区、促销活动等。\n4. 业务查询']
 ['1. 中国联通手机营业厅网页版wap.10010.com是面向普通手机上网用户、可提供7×24小时服务的营销平台。\n2. 您只需在手机浏览器上输入wap.10010.com，凭手机号码和服务密码，即可轻松访问中国联通手机营业厅。']
 ['1. 下载手机登录www.10010.com下载的用户有iPhone版、iPad版、Android版、乐Phone版及Symbian版共五个版本的客户端。\n2. 不同手机品牌、不同用户类型（后付费/预付费）、不同网络用户（2G/3G用户）看到的版本和可使用的功能会有所差别。']
 ['1. 当月话费用户登录客户端后，点击当月话费按键，可显示用户当月话费情况套餐余量用户登录客户端后，点击套餐余量按键，可显示用户当月套餐使用情况历史账单用户登录客户端后，点击历史账单按键，可显示用户历史账单使用情况通话详单用户登录客户']
 ['1.']
 ['1. 中国联通电子渠道的部分功能需要您输入手机号码和服务密码，如您忘记服务密码，可通过以下方式重新设置。请问，重新设置密码的方式有哪些？\n2. 通过手机营业厅在手机浏览器上输入wap.10010.com登录中国联通手机营业厅—>进入登录页面—>点击“忘记密码”—>']
 ['1. 网络的定义是指第三代移动通信技术，是将无线通信与互联网等多媒体通信结合的新一代移动通信系统。\n2. 中国联通采用的是通信标准。']
 ['1. 全球开通网络最多是采用制式的。\n2. 全球有个国家具有网络覆盖全球市场。\n3. 全球市场占有率最高的终端款式是支持商用终端的款式。\n4. 支持商用终端的款式数量最多是款。']
 ['1. 不需要;\n2. 不需要;\n3. 不需要.']
 ['1.后付费用户需要签约和登记用户信息；先使用，后付费，按月付费，享受客户俱乐部、积分等客户服务。可以对后付费用户进行信用额度操纵']
 ['1. 标准产品〔预付费〕是实时扣费、先付费后使用的产品。\n2. 先付费后使用的产品是实时扣费、

In [10]:
df = df.dropna().reset_index().drop('index',axis=1)
df.to_csv("docx/qa/qa-chinese-gpt.csv", index=False)

## Debug proxy 

In [16]:
test_context = df.context[0]
response = openai.Completion.create(
            engine="davinci-instruct-beta-v3",
            prompt=f"请根据以下文本生成问题\n\n文本: {test_context}\n\n问题:\n1.",
            temperature=0,
            max_tokens=800,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )


In [17]:
print(response['choices'][0]['text'])

 中国联通采用的是什么标准?
2. 网络的定义是指第三代移动通信技术，是将无线通信与互联网等多媒体通信结合的新一代移动通信系统，能够提供包括可视   、无线上网，手机上网、手机电视、手机音乐等多种信息服务。

